

## 🌐 1️⃣ SQLite with `sqlite3` (Direct SQL Queries)

**Folder structure:**

```
project/
│   app.py
└───templates/
      users.html
```

**app.py**

```python
from flask import Flask, render_template
import sqlite3

app = Flask(__name__)

# Initialize database and create table
def init_db():
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    c.execute("""CREATE TABLE IF NOT EXISTS users (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    name TEXT NOT NULL,
                    email TEXT NOT NULL
                )""")
    conn.commit()
    conn.close()

@app.route("/add_user/<name>/<email>")
def add_user(name, email):
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    c.execute("INSERT INTO users (name, email) VALUES (?, ?)", (name, email))
    conn.commit()
    conn.close()
    return f"User {name} added!"

@app.route("/users")
def users():
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    c.execute("SELECT * FROM users")
    users_list = c.fetchall()
    conn.close()
    return render_template("users.html", users=users_list)

if __name__ == "__main__":
    init_db()
    app.run(debug=True)
```

**templates/users.html**

```html
<!DOCTYPE html>
<html>
<head>
    <title>Users</title>
</head>
<body>
    <h1>All Users</h1>
    <ul>
        {% for user in users %}
            <li>{{ user[1] }} ({{ user[2] }})</li>
        {% endfor %}
    </ul>
</body>
</html>
```

✅ **Key Points:**

* `sqlite3.connect("users.db")` → connects to SQLite file
* `cursor.execute()` → run SQL queries
* `fetchall()` → retrieve data
* Direct SQL works well for small projects, but code can become messy for large apps

---

## 🌐 2️⃣ SQLite with SQLAlchemy ORM (Recommended)

**Install SQLAlchemy**

```bash
pip install Flask-SQLAlchemy
```

**app.py**

```python
from flask import Flask, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

# Model
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(50), nullable=False)
    email = db.Column(db.String(120), nullable=False)

    def __repr__(self):
        return f"<User {self.name}>"

@app.before_first_request
def create_tables():
    db.create_all()

@app.route("/add_user/<name>/<email>")
def add_user(name, email):
    new_user = User(name=name, email=email)
    db.session.add(new_user)
    db.session.commit()
    return f"User {name} added!"

@app.route("/users")
def users():
    users_list = User.query.all()
    return render_template("users.html", users=users_list)

if __name__ == "__main__":
    app.run(debug=True)
```

**templates/users.html**

```html
<ul>
    {% for user in users %}
        <li>{{ user.name }} ({{ user.email }})</li>
    {% endfor %}
</ul>
```

✅ **Advantages of SQLAlchemy ORM:**

* Work with **Python objects instead of raw SQL**
* Cleaner and maintainable code
* Easy **CRUD operations** (Create, Read, Update, Delete)
* Integrates smoothly with **Flask-WTF forms** and templates

---

### 🔎 Key Takeaways

| Feature         | sqlite3 (Direct)      | SQLAlchemy (ORM)                                   |
| --------------- | --------------------- | -------------------------------------------------- |
| Code structure  | Raw SQL queries       | Python classes & objects                           |
| Ease of use     | Simple for small apps | Cleaner for large apps                             |
| CRUD operations | Manual SQL            | Methods like `.add()`, `.commit()`, `.query.all()` |
| Validation      | Manual                | Can combine with Flask-WTF                         |

---

